In [2]:
import os
import pickle
import sys
import math
sys.path.append("../")
from od.env.config import SUMO_SIM_SECONDS,SUMO_TOTAL_SECONDS
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,ROOT_DIR
from od.env.station import BS_PRESET
from od.social.manager.types import DynamicSocialGroupBehaviour
from od.env.config import NET_TIMEOUT_SECONDS
import od.misc.statistic as oms

In [3]:
print(os.getcwd())
interest_config = InterestConfig(DynamicSocialGroupBehaviour.MAX_N_MEMBER,20,False,ResourceAllocatorType.NOMA_APR,True,1.4,2)
statistic = None
print(interest_config)
with open("../data/GERNA/{}/object.pickle".format(interest_config.folder()), "rb") as file:
    statistic = pickle.load(file)
statistic.nft_traffic = {}
statistic.raw_sg_header = statistic.sg_header

/home/OD/repos/ProjectSumo/ipynb
MAX_N_MEMBER(20)-nQoS-yRSU-NOMA_APR-D1.4-S2


In [4]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV


In [5]:
c2g_app_stats = statistic.ExtractNetworkTraffic(NetFlowType.C2G,True)
cri_app_stats = statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL,True)

In [6]:
c2g_app_stats = statistic.ExtractNetworkTraffic(NetFlowType.C2G,True)
cri_app_stats = statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL,True)
for name in c2g_app_stats.keys():
    c2g_app_stats[name].time_veh_serv = cri_app_stats[name].time_veh_serv
    c2g_app_stats[name].time_veh_tx = cri_app_stats[name].time_veh_tx
    c2g_app_stats[name].time_veh_txq = cri_app_stats[name].time_veh_txq

In [7]:
list(c2g_app_stats.values())[-1].time_veh_serv
list(c2g_app_stats.values())[-1].time_veh_txq

IndexError: list index out of range

In [8]:
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2, UMI-1, UMI-2, UMI-3, UMI-4, UMI-5]

In [9]:
TARGET_NFT = NetFlowType.GENERAL

In [10]:
app_stats = statistic.ExtractNetworkTraffic(TARGET_NFT,True)

In [11]:
statistic.VehicleReceivedIntactAppdataReport(app_stats)

{'avg': 0.4294412617295618,
 'max': 7.798000000000002,
 'min': 0.001999999999981128}

In [12]:
statistic.AppdataE2EReport(app_stats)

{'avg': 0.6916970039925935,
 'max': 7.798000000000002,
 'min': 0.001999999999981128}

In [13]:
statistic.AppdataTXQReport(app_stats)

{'avg': 0.6892272312867462, 'max': 7.792999999999978, 'min': 0.0}

In [14]:
record_id = list(app_stats.keys())
data = app_stats[record_id[0]]

In [15]:
data.time_veh_serv

186.0

In [16]:
data.time_veh_tx

[[186.0, 186.001]]

In [17]:
data.time_veh_txq

[[186.0, 186.0]]

In [18]:
statistic.AppdataTXQReport(app_stats)

{'avg': 0.6892272312867462, 'max': 7.792999999999978, 'min': 0.0}

In [19]:
for name,record in app_stats.items():
    singleton = {name:record}
    e2e = statistic.AppdataE2EReport(singleton)["avg"]
    wait =  statistic.AppdataTXQReport(singleton)["avg"]
    if( e2e < wait):
        print(e2e,wait)
        print(name,record)
        break

In [20]:
name = "flow_1.6-222"
for sg in statistic.social_group:
    if(name in statistic.sg_header[sg]):
        print(sg,name)
        record = statistic.sg_header[sg][name]
print("at",record.at)
print("trip:",record.time_veh_trip)
print("veh serv:",record.time_veh_serv)
print("veh tx:",record.time_veh_tx)
print("veh txq:",record.time_veh_txq)
print("bs tx:",record.time_bs_tx)
print("bs txq:",record.time_bs_txq)
print("bs drop:",record.time_bs_drop)
print("bs serv:",record.time_bs_serv)
print("is src ot:",record.is_src_ot)
print("is bs ot:",record.is_bs_ot)


DYN_SG_3 flow_1.6-222
at 187.3
trip: {'flow_0.35': 0.23499999999998522, 'flow_0.36': 0.23499999999998522, 'flow_0.42': 0.23499999999998522, 'flow_0.43': 0.23499999999998522, 'flow_0.45': 0.23499999999998522, 'flow_1.8': 0.23499999999998522, 'flow_13.3': 0.23499999999998522, 'flow_2.35': 0.23499999999998522, 'flow_2.39': 0.23499999999998522, 'flow_2.41': 0.23499999999998522, 'flow_2.48': 0.23499999999998522, 'flow_2.51': 0.23499999999998522, 'flow_2.52': 0.23499999999998522, 'flow_2.53': 0.23499999999998522, 'flow_8.4': 0.23499999999998522}
veh serv: 187.533
veh tx: [[187.533, 187.534]]
veh txq: [[187.3, 187.533]]
bs tx: [[[187.534, 187.535]], [[195.097, 195.098], [195.098, 195.099]], [], [], [], [], []]
bs txq: [[[187.534, 187.534]], [[187.534, 195.097], [195.098, 195.098], [195.0, 195.0]], [], [], [], [], []]
bs drop: [-1, 195, -1, -1, -1, -1, -1]
bs serv: [187.534, -1, -1, -1, -1, -1, -1]
is src ot: False
is bs ot: [False, True, False, False, False, False, False]


In [28]:
for sg in statistic.social_group:
    print(sg,statistic.AppdataTimeoutRatioReport(statistic.sg_header[sg]))

CRASH 0.0
RCWS 0.0
DYN_SG_0 0.0
DYN_SG_1 0.0
DYN_SG_2 0.0
DYN_SG_3 0.5528688524590164
DYN_SG_4 0.0
DYN_SG_5 0.0
DYN_SG_6 0.0
DYN_SG_7 0.0
DYN_SG_8 0.0
DYN_SG_9 0.0


In [29]:
# preprocess overtimes
for sg in statistic.social_group:
    for record in statistic.sg_header[sg].values():
        for bs in GV.NET_STATION_CONTROLLER:
            if(len(record.time_bs_txq[bs]) > 0  and record.time_bs_txq[bs][-1][0] == SUMO_TOTAL_SECONDS and record.time_bs_txq[bs][-1][1] == SUMO_TOTAL_SECONDS and record.time_bs_drop[bs]==SUMO_TOTAL_SECONDS):
                record.time_bs_drop[bs] = -1
                record.time_bs_txq[bs].pop()
                record.time_bs_txq[bs].append([record.time_bs_tx[bs][-1][1],SUMO_TOTAL_SECONDS+1])
            if(len(record.time_bs_txq[bs]) > 0  and record.time_bs_txq[bs][-1][1] == SUMO_TOTAL_SECONDS):
                record.time_bs_txq[bs][-1][1] = SUMO_TOTAL_SECONDS + 1


In [1]:
print(type(statistic.sg_header[sg]))

for sg in statistic.social_group:
    print(statistic.AppdataTimeoutRatioReport(statistic.sg_header[sg]))

NameError: name 'statistic' is not defined

In [ ]:
flow_1.6-222